In [ ]:
import asyncio
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, FallbackChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

import json

In [2]:
import json
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
load_dotenv()

True

In [3]:
print("DB_PORT: ", os.getenv("DB_PORT"))
print("DB_HOST: ", os.getenv("DB_HOST"))
print("DB_USER: ", os.getenv("DB_USER"))
print("DB_PASSWORD: ", os.getenv("DB_PASSWORD"))
print("DB_NAME: ", os.getenv("DB_NAME"))


DB_PORT:  5432
DB_HOST:  localhost
DB_USER:  tntuser
DB_PASSWORD:  tntdemo
DB_NAME:  demo


In [4]:
from decimal import Decimal

class DatabaseConnector:
    def __init__(self):
        self.create_connection()

    @kernel_function(description="Create a connection object to the postgres database.")
    def create_connection(self):
        print("create_connection function called... ")
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        self.connection = connection
        self.cursor = self.connection.cursor(cursor_factory=RealDictCursor)

    @kernel_function(description="Fetches user information for a given account id.")
    def query_database(self, query: Annotated[str, "query to be executed"]) -> Annotated[str, "Returns the queried information as a json"]:
        """
        Fetches the information from the required table in PostgreSQL database.

        :param query (str): the query to be executed.
        :return: fetched information as a JSON string.
        :rtype: str
        """
        print("query_database function called... query: ", query)
        try:
            cursor = self.cursor
            cursor.execute(query=query)
            result_record = cursor.fetchone()
            if result_record:
                # Convert Decimal values to strings
                for key, value in result_record.items():
                    if isinstance(value, Decimal):
                        result_record[key] = str(value)
                return json.dumps({"result_record": result_record})
            else:
                return json.dumps({"error": "An error occured while fetching the data."})
        except Exception as e:
            return json.dumps({"error": str(e)})
        
        
    
    @kernel_function(description="Closes the connection to the database.")
    def close_connection(self) -> Annotated[str, "Returns a message indicating the status of the connection closure."]:
        """
        Closes the connection to the PostgreSQL database.

        :return: Message indicating the status of the connection closure.
        :rtype: str
        """
        print("close_connection function called... ")
        try:
            if self.connection:
                self.cursor.close()
                self.connection.close()
            return "Connection closed successfully."
        except Exception as e:
            return str(e)

In [5]:
accounts_instance = DatabaseConnector()

create_connection function called... 


In [6]:
accounts_instance.create_connection()

create_connection function called... 


In [7]:
accounts_instance.query_database("SELECT * FROM accounts WHERE account_id = 1")

query_database function called... query:  SELECT * FROM accounts WHERE account_id = 1


'{"result_record": {"account_id": 1, "name": "HBL", "email": "abcd@gef.com", "phone": "9876543210", "balance": "99034.50"}}'

In [8]:
accounts_instance.close_connection()

close_connection function called... 


'Connection closed successfully.'

In [9]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello",
    "Could you please give me more information about the account id 1 ?",
    "No more questions. Thank you!",
]


In [10]:
USER_INPUTS = [
    "Hello",
    "Could you please find the account with maximum balance? Display only the name and balance of the account.",
    "No more questions. Thank you!",
]

In [ ]:
# 1. Create the instance of the Kernel to register the plugin and service
service_id = "agent" 
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
kernel = Kernel()

kernel.add_plugin(DatabaseConnector(), plugin_name="db_plugin")
kernel.add_service(
            AzureChatCompletion(service_id=service_id,
                                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                
            ))                        

create_connection function called... 


In [12]:
# 2. Configure the function choice behavior to auto invoke kernel functions
# so that the agent can automatically execute the Database plugin functions when needed
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# for parallel executions of the agent
# options = FunctionChoiceBehaviorOptions(allow_concurrent_invocation=True)
# settings.function_choice_behavior = FunctionChoiceBehavior.Auto(options)


In [ ]:
# 3. Create the agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="Host",
    instructions="""You are a helpful Software Engineer with expertise in SQL queries. Answer the user question by retrieving required information from database tables. 
    Account information is stored in the 'accounts' table and the schema is as follows:
    account_id: int, name: text, balance: decimal, email: text, phone: text, address: text
    
    The user will ask questions about the account information by providing the account_id.
    
    Write the response in a clear and concise manner in a short paragraph.
    Remember to create the connection first and then query the database as needed. 
    After the customer confirms there are no more questions, close the Database connection""",
    arguments=KernelArguments(settings=settings), 
)


In [14]:
# 4. Create a chat history to hold the conversation
chat_history = ChatHistory()

In [15]:
for user_input in USER_INPUTS:
    # 5. Add the user input to the chat history
    chat_history.add_user_message(user_input)
    print(f"# User: {user_input}")
    # 6. Invoke the agent for a response
    async for content in agent.invoke(messages=chat_history):
        print(f"# {content.name}: ", end="")
        if (
            not any(isinstance(item, (FunctionCallContent, FunctionResultContent)) for item in content.items)
            and content.content
        ):
            # We only want to print the content if it's not a function call or result
            print(f"{content.content}", end="", flush=True)
    print("")  # Print a newline to separate the messages

# User: Hello
# Host: Hello! How can I assist you with your account information today?
# User: Could you please find the account with maximum balance? Display only the name and balance of the account.
create_connection function called... 
query_database function called... query:  SELECT name, balance FROM accounts ORDER BY balance DESC LIMIT 1;
# Host: The account with the maximum balance belongs to "Vikas" with a balance of ₹123,456.00. Let me know if you have any further questions!
# User: No more questions. Thank you!
create_connection function called... 
query_database function called... query:  SELECT name, balance FROM accounts ORDER BY balance DESC LIMIT 1;
close_connection function called... 
# Host: The account with the maximum balance belongs to "Vikas" with a balance of 123,456.00. If you have more questions in the future, feel free to ask. Thank you!
